In [2]:
import pandas as pd

import fsspec

fs = fsspec.filesystem(
    'filecache',
    target_protocol='s3',
    target_options={
        'key': 'YCAJEZ9PJ6itAHdK91ZPoWAp1',
        'secret': 'YCMesgjgH3dgexMVchUF5d9XRzBqYJGFllsUTP3v',
        'client_kwargs': {'endpoint_url': 'https://storage.yandexcloud.net'}
    },
    cache_storage='data',
    check_files=True)

In [19]:
with fs.open('s3://snap-simfin/us-shareprices-daily-2022-10-20.zip') as f:
    prices = pd.read_csv(f, compression='zip', sep=';')
    
prices['price_scaling'] = prices.Close.pct_change() + 1
prices['div_scaling'] = (prices.Dividend / prices.Close) + 1
prices['scaling'] = prices.price_scaling * prices.div_scaling
prices['scaling_c'] = prices.groupby('SimFinId').scaling.cumprod()
prices['Date'] = pd.to_datetime(prices.Date)

In [62]:
sp500tr = pd.read_csv('sp500tr.csv', sep=';', header=None, names=['sp_date', 'value'])
sp500tr['sp_date'] = pd.to_datetime(sp500tr.sp_date, dayfirst=True)
sp500tr['scaling'] = sp500tr.value.pct_change() + 1
sp500tr['sp_scaling_c'] = sp500tr.scaling.cumprod()

In [64]:
prices = prices.merge(sp500tr[['sp_date', 'sp_scaling_c']], left_on='Date', right_on='sp_date')